## Collaborative filtering and Hybird

In [154]:
import numpy as np
import pandas as pd
from surprise import Dataset, SVD ,Reader
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from surprise.model_selection import cross_validate , KFold 
from surprise import model_selection

In [155]:
moviesData = pd.read_csv("../data/MoviesMetadata.csv")
moviesData.head(10)

,budget,id,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,...,title,vote_average,vote_count,name_genres,id_genres,name_production_countries,iso_3166_1_production_countries,name_production_companies,id_production_companies,year
0,30.0,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,373.554033,...,Toy Story,7.7,5415.0,"Animation, Comedy, Family","16, 35, 10751",United States of America,US,Pixar Animation Studios,3,1995
1,65.0,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,262.797249,...,Jumanji,6.9,2413.0,"Adventure, Fantasy, Family","12, 14, 10751",United States of America,US,"TriStar Pictures, Teitler Film, Interscope Com...","559, 2550, 10201",1995
2,0.0,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.712900,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,1995-12-22,0.000000,...,Grumpier Old Men,6.5,92.0,"Romance, Comedy","10749, 35",United States of America,US,"Warner Bros., Lancaster Gate","6194, 19464",1995
3,16.0,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,1995-12-22,81.452156,...,Waiting to Exhale,6.1,34.0,"Comedy, Drama, Romance","35, 18, 10749",United States of America,US,Twentieth Century Fox Film Corporation,306,1995
4,0.0,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,1995-02-10,76.578911,...,Father of the Bride Part II,5.7,173.0,Comedy,35,United States of America,US,"Sandollar Productions, Touchstone Pictures","5842, 9195",1995
5,60.0,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",17.924927,/zMyfPUelumio3tiDKPffaUpsQTD.jpg,1995-12-15,187.436818,...,Heat,7.7,1886.0,"Action, Crime, Drama, Thriller","28, 80, 18, 53",United States of America,US,"Regency Enterprises, Forward Pass, Warner Bros.","508, 675, 6194",1995
6,58.0,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,6.677277,/jQh15y5YB7bWz1NtffNZmRw0s9D.jpg,1995-12-15,0.000000,...,Sabrina,6.2,141.0,"Comedy, Romance","35, 10749","Germany, United States of America","DE, US","Paramount Pictures, Scott Rudin Productions, M...","4, 258, 932, 5842, 14941, 55873, 58079",1995
7,0.0,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",2.561161,/sGO5Qa55p7wTu7FJcX4H4xIVKvS.jpg,1995-12-22,0.000000,...,Tom and Huck,5.4,45.0,"Action, Adventure, Drama, Family","28, 12, 18, 10751",United States of America,US,Walt Disney Pictures,2,1995
8,35.0,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,5.231580,/eoWvKD60lT95Ss1MYNgVExpo5iU.jpg,1995-12-22,64.350171,...,Sudden Death,5.5,174.0,"Action, Adventure, Thriller","28, 12, 53",United States of America,US,"Universal Pictures, Imperial Entertainment, Si...","33, 21437, 23770",1995
9,58.0,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,14.686036,/5c0ovjT41KnYIHYuF4AWsTe3sKh.jpg,1995-11-16,352.194034,...,GoldenEye,6.6,1194.0,"Adventure, Action, Thriller","12, 28, 53","United Kingdom, United States of America","GB, US","United Artists, Eon Productions","60, 7576",1995


In [156]:
reader = Reader()
ratings = pd.read_csv('../data/ratings_small.csv')
ratings

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [157]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
svd = SVD()
model_selection.validation.cross_validate(svd, data, measures=['RMSE', 'MAE'])

{'test_rmse': array([0.9003493 , 0.89498773, 0.89158641, 0.90078617, 0.89396564]),
 'test_mae': array([0.69559345, 0.68850157, 0.68452703, 0.69126755, 0.69069392]),
 'fit_time': (0.8176331520080566,
  0.8362753391265869,
  0.7632372379302979,
  0.7718489170074463,
  0.7094261646270752),
 'test_time': (0.5366709232330322,
  0.06749081611633301,
  0.06446003913879395,
  0.06383585929870605,
  0.06429100036621094)}

In [158]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
svd = SVD()
trainset = data.build_full_trainset()
svd.fit(trainset)

In [159]:
svd.predict(1 ,31)

Prediction(uid=1, iid=31, r_ui=None, est=2.339840626360479, details={'was_impossible': False})

In [160]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [161]:
links = pd.read_csv('../data/links.csv')
links = links.dropna()
links.isna().sum()
links['tmdbId'] = links['tmdbId'].astype('int64')
type(links['tmdbId'][0])

numpy.int64

In [162]:
links

,movieId,imdbId,tmdbId
0,1,114709,862
1,2,113497,8844
2,3,113228,15602
3,4,114885,31357
4,5,113041,11862
...,...,...,...
45838,176269,6209470,439050
45839,176271,2028550,111109
45840,176273,303758,67758
45841,176275,8536,227506


In [163]:
movie_com = pd.read_csv('../data/MovieBasedRecommender.csv')
movie_com

,model_feature,title,score,id
0,jealousi toy boy friendship friend rivalri boy...,Toy Story,0.357760,862
1,boardgam disappear basedonchildren'sbook newho...,Jumanji,0.284882,8844
2,fish bestfriend duringcreditssting oldmen walt...,Grumpier Old Men,0.206639,15602
3,basedonnovel interracialrelationship singlemot...,Waiting to Exhale,0.178489,31357
4,babi midlifecrisi confid age daughter motherda...,Father of the Bride Part II,0.180003,11862
...,...,...,...,...
45448,tragiclov leilahatami kouroshtahami elhamkorda...,Subdue,0.166561,439050
45449,artist play pinoy angelaquino perrydizon hazel...,Century of Birthing,0.172627,111109
45450,erikaeleniak adambaldwin juliedupage jamesrema...,Betrayal,0.162892,67758
45451,iwanmosschuchin nathalielissenko pavelpavlov a...,Satan Triumphant,0.167313,227506


In [164]:
movie_com['model_feature'] = movie_com['model_feature'].fillna('')
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(movie_com['model_feature'])

In [165]:
tfidf_matrix.shape

(45453, 943773)

In [166]:
cosine_sim = linear_kernel(tfidf_matrix[:15000], tfidf_matrix)

In [167]:
indices = pd.Series(moviesData.index, index=moviesData['title'])

In [168]:
movie_id = links[['movieId', 'tmdbId']]
movie_id.to_csv('../data/movie_id.csv', index=False)

In [176]:
def hybrid(userId, title):
    idx = indices[title]
    try:
        sim_scores = list(enumerate(cosine_sim[int(idx[0])]))
    except:
        sim_scores = list(enumerate(cosine_sim[int(idx)]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    movies = movie_com.iloc[movie_indices][['title', 'id']]
    tempLinks = links[links['tmdbId'].isin(movies['id'].tolist())]  
    l = list() 
    
    for item in  tempLinks["movieId"]:
            l.append (svd.predict(userId, item).est)
    
    tempData = pd.DataFrame({"title" :movies['title'],"est": l, "id" :movies['id']})
    tempData = tempData.sort_values('est', ascending=False)
    tempData = tempData[tempData["est"] >= 2.5]

    return tempData.head(10)

In [177]:
hybrid(1, 'Toy Story')

,title,est,id
25804,Toy Story That Time Forgot,3.195494,256835
3001,Toy Story 2,3.033069,863
21936,Toy Story of Terror!,3.018621,213121
25802,Partysaurus Rex,2.822334,130925
10675,Luxo Jr.,2.743323,13925
24526,Hawaiian Vacation,2.662705,77887
40196,Uncle P,2.662705,65755
27794,Radiopiratene,2.662705,60769
12986,Dead Fury,2.662705,37776
45124,Banana,2.662705,54551


In [171]:
def getUserRecommender(userID):
    tempRatings = ratings[ratings['userId'] == userID]
    tempRatings =  tempRatings[tempRatings["rating"] >= 4]

    tempLinks = links[links['tmdbId'].isin(tempRatings['movieId'].tolist())]  
    titlesData = moviesData[moviesData["id"].isin(tempLinks["tmdbId"])]
    
    resultDataFrame = pd.DataFrame()
        
    for title in titlesData["title"]:
        resultDataFrame = pd.concat([hybrid(userID,title), resultDataFrame], ignore_index = True)
        
    return resultDataFrame.sort_values('est', ascending=False)

In [178]:
getUserRecommender(100)

,title,est,id
40,Frank Herbert's Dune,4.051149,876
41,Dumbland,3.974804,201198
20,Mac,3.633089,111883
0,Beautiful Boxer,3.620764,6414
50,Mo' Better Blues,3.583589,41823
51,If God Is Willing and da Creek Don't Rise,3.570890,96451
52,Jerrod Carmichael: Love at the Store,3.567517,295891
30,"The Book of Mormon Movie, Volume 1: The Journey",3.510747,48382
31,Sethupathi,3.478012,369834
1,Shamitabh,3.454191,323426
